## Oliwier Misztal   --------------   December 2, 2025
## Arthur Murphy
## Javi Jorge
# Molecular Modeling - Monte Carlo Methods

In [37]:
import numpy as np
from numba import njit
import time

# At first, we implement functions that are compiled with Numba for speed.

@njit
def create_nbr(L):
    """
    Creates the neighbor table for a 2D square lattice.
    Returns an array of shape (N, 4).
    """
    N = L * L
    nbr = np.zeros((N, 4), dtype=np.int64)
    
    # Pre-compute shifts (Periodic Boundaries)
    ip = np.arange(L) + 1
    im = np.arange(L) - 1
    ip[L - 1] = 0
    im[0] = L - 1
    
    for y in range(L):
        for x in range(L):
            i = x + y * L
            nbr[i, 0] = ip[x] + y * L   # Right neighbor
            nbr[i, 1] = im[x] + y * L   # Left neighbor
            nbr[i, 2] = x + ip[y] * L   # Up neighbor
            nbr[i, 3] = x + im[y] * L   # Down neighbor
            
    return nbr

@njit
def precompute_exponentials(T, nbr):
    """
    Analyzes the neighbor table to find 'z' (coordination number)
    and pre-computes the exponential table for all possible
    positive Delta E values.
    """
    beta = 1.0 / T
    
    # We analyze neighbors (nbr) to find 'z'
    # We assume a regular lattice where every site has the same number of neighbors.
    z = nbr.shape[1]  
    
    # We determine maximum possible energy change
    # Maximum change happens when flipping a spin surrounded by aligned neighbors.
    # Delta E = 2 * s_i * sum(neighbors). Max sum is z.
    # Max Delta E = 2 * 1 * z = 2z
    max_delta_E = 2 * z
    
    # We create the table
    # We need indices up to max_delta_E. 
    exp_table = np.zeros(max_delta_E + 1, dtype=np.float64)
    
    # We fill valid Delta E values
    # The sum of neighbors (h_i) goes from z down to -z in steps of 2.
    # We only care about POSITIVE Delta E (where we need the probability).
    # This corresponds to neighbor sums: z, z-2, z-4 ... > 0
    current_sum = z
    while current_sum > 0:
        delta_E = 2 * current_sum
        exp_table[delta_E] = np.exp(-beta * delta_E)
        current_sum -= 2
        
    return exp_table

@njit
def metropolis_step(state, nbr, exp_table):
    """
    Performs 1 Full Monte Carlo Step (N attempted flips).
    This is highly optimized machine code loop.
    """
    N = state.shape[0]
    z = nbr.shape[1] # Number of neighbors
    
    # Loop N times (1 MCS)
    for _ in range(N):
        # Pick random site
        i = np.random.randint(0, N)
        s_i = state[i]
        
        # Calculate sum of neighbors (Generic loop for any z)
        h_i = 0
        for k in range(z):
            h_i += state[nbr[i, k]]
        
        # Energy change
        delta_E = 2 * s_i * h_i
        
        # Metropolis Acceptance
        if delta_E <= 0:
            state[i] = -s_i # Accept & Flip
        else:
            # Look up probability
            if np.random.random() < exp_table[delta_E]:
                state[i] = -s_i # Accept & Flip
                
    return state

@njit
def measure_observables(state, nbr):
    """
    Calculates total Magnetization (M) and Energy (E) of the current state.
    """
    N = state.shape[0]
    z = nbr.shape[1]
    M = 0
    E = 0.0
    for i in range(N):
        M += state[i]
        
        # Calculate sum of neighbors for energy
        h_i = 0
        for k in range(z):
            h_i += state[nbr[i, k]]
            
        # E = - sum(s_i * s_j). 
        # We divide by 2.0 at the end to correct for double counting.
        E -= state[i] * h_i
    return E / 2.0, M

# Main Simulation Function

def run_simulation(L=100, T=2.27, mcs_steps=10000, n_meas=100, spins=None):
    # Setup
    N = L * L
    
    # Geometry
    nbr = create_nbr(L)
    
    # Smart Precalculation
    # This automatically detects if z=4 (square), z=6 (triangular), etc.
    exp_table = precompute_exponentials(T, nbr)

    # Pre-allocate arrays for measurements
    num_measurements = mcs_steps // n_meas
    energies = np.zeros(num_measurements, dtype=np.float64)
    magnetizations = np.zeros(num_measurements, dtype=np.int64)
    
    # Initialize Spins if not provided
    if spins is None:
        spins = np.random.choice(np.array([-1, 1], dtype=np.int8), size=N)
    
    # We need to JIT compile the functions before timing
    # So we run once with dummy data to force Numba to compile
    print("Compiling functions...")
    _ = metropolis_step(spins.copy(), nbr, exp_table)
    print("Compilation complete. Starting simulation...")
    
    # Start Timing
    start_time = time.time()
    
    # Run the optimized loop
    for step in range(mcs_steps):
        metropolis_step(spins, nbr, exp_table)
        # Measurement
        if step % n_meas == 0:
            idx = step // n_meas - 1
            E, M = measure_observables(spins, nbr)
            energies[idx] = E
            magnetizations[idx] = M
        
    end_time = time.time()
    elapsed = end_time - start_time
    
    # Statistics
    total_flips = N * mcs_steps
    flips_per_sec = total_flips / elapsed
    
    print(f"L={L}, T={T}, MCS={mcs_steps}")
    print(f"Time elapsed: {elapsed:.4f} s")
    print(f"Speed: {flips_per_sec:.2e} updates/second")
    
    return spins, energies, magnetizations

if __name__ == "__main__":
    # Test with 3 example files:
     #
    run_simulation(L=100, T=2.27, mcs_steps=5000)


Compiling functions...
Compilation complete. Starting simulation...
L=100, T=2.27, MCS=5000
Time elapsed: 4.7587 s
Speed: 1.05e+07 updates/second


In [38]:
def read_configuration(filename):
    data = np.loadtxt(filename, dtype=int)
    spins = data[:, 1]
    return spins

In [39]:
def initialize_systems(L, initial_state='random', filename=None):
    N = L * L
    if initial_state == 'random':
        spins = np.random.choice(np.array([-1, 1], dtype=np.int8), size=N)
    elif initial_state == 'file' and filename is not None:
        spins = read_configuration(filename)
    else:
        raise ValueError("Invalid initial state or filename not provided.")
    return spins

In [40]:
import numpy as np
def calc_energy_vectorized(state, nbr):
    energy = -np.sum(state[nbr] * state[:, None])
    return energy/2

In [ ]:
L = 10
nbr = create_nbr(L)
N = L * L
spins_test_configuration = initialize_systems(L, initial_state='file', filename='test_configuration.dat')
spins_mys1 = initialize_systems(L, initial_state='file', filename='mys1.dat')
spins_mys2 = initialize_systems(L, initial_state='file', filename='mys2.dat')
spins_mys3 = initialize_systems(L, initial_state='file', filename='mys3.dat')

print(measure_observables(spins_test_configuration, nbr))
start_time = time.time()
print(measure_observables(spins_test_configuration, nbr))
print(measure_observables(spins_mys1, nbr))
print(measure_observables(spins_mys2, nbr))
print(measure_observables(spins_mys3, nbr))
end_time = time.time()
print(f"Time taken for measure_observables: {end_time - start_time:.6f} seconds")

start_time = time.time()
print(calc_energy_vectorized(spins_test_configuration, nbr))
print(calc_energy_vectorized(spins_mys1, nbr))
print(calc_energy_vectorized(spins_mys2, nbr))
print(calc_energy_vectorized(spins_mys3, nbr))
end_time = time.time()
print(f"Time taken for calc_energy_vectorized: {end_time - start_time:.6f} seconds")
    

(-8.0, 8)
(-8.0, 8)
(8.0, -4)
(0.0, -4)
(-4.0, -6)
Time taken for measure_observables: 0.000782 seconds
-8.0
8.0
0.0
-4.0
Time taken for calc_energy_vectorized: 0.000692 seconds


In [53]:
L = 1000
nbr = create_nbr(L)
N = L * L
spins_test_configuration = initialize_systems(L, initial_state="random")

print(measure_observables(spins_test_configuration, nbr))
start_time = time.time()
print(measure_observables(spins_test_configuration, nbr))
end_time = time.time()
print(f"Time taken for measure_observables: {end_time - start_time:.6f} seconds")

start_time = time.time()
print(calc_energy_vectorized(spins_test_configuration, nbr))
end_time = time.time()
print(f"Time taken for calc_energy_vectorized: {end_time - start_time:.6f} seconds")
    

(-868.0, 1990)
(-868.0, 1990)
Time taken for measure_observables: 0.015276 seconds
-868.0
Time taken for calc_energy_vectorized: 0.040495 seconds


In [42]:
spins_last, energies, magnetizations = run_simulation(L=10, T=2.27, mcs_steps=5000, n_meas=100, spins=spins_test_configuration)

Compiling functions...
Compilation complete. Starting simulation...
L=10, T=2.27, MCS=5000
Time elapsed: 0.5104 s
Speed: 9.80e+05 updates/second


In [ ]:
def binning_analysis(data, max_bin_size=None):
    """
    Perform binning analysis on time series data.
    
    Parameters:
        data: (n_measurements,) time series array
        max_bin_size: maximum bin size (~100 bins should remain)
    
    Returns:
        bin_sizes: array of bin sizes (powers of 2)
        bin_errors: standard error for each bin size
        bin_means: mean value from binned data
    """
    n_data = len(data)
    
    # Max bin size such that we have ~100 bins left
    if max_bin_size is None:
        max_bin_size = max(1, n_data // 100)
    
    # Bin sizes: 1, 2, 4, 8, 16, 32, ...
    bin_sizes = []
    bin_errors = []
    bin_means = []
    
    m = 1
    while m <= max_bin_size:
        n_bins = n_data // m
        
        if n_bins < 1:
            break
        
        # Bin the data
        binned_data = np.array([np.mean(data[i*m:(i+1)*m]) for i in range(n_bins)])
        
        # Calculate statistics
        bin_mean = np.mean(binned_data)
        bin_std = np.std(binned_data, ddof=1)  # Sample std dev
        bin_error = bin_std / np.sqrt(n_bins)  # Standard error of the mean
        
        bin_sizes.append(m)
        bin_errors.append(bin_error)
        bin_means.append(bin_mean)
        
        m *= 2
    
    return np.array(bin_sizes), np.array(bin_errors), np.array(bin_means)

def read_energies(filename):
    data = np.loadtxt(filename)
    energies = data[:]
    return energies

energies = read_energies('bintest1.dat')
# Perform binning analysis
bin_sizes, energy_errors, energy_means = binning_analysis(energies)
bin_sizes_m, mag_errors, mag_means = binning_analysis(np.abs(magnetizations))

print(f"{'='*70}")
print(f"TASK 3: BINNING ANALYSIS")
print(f"{'='*70}")
print(f"\nEnergy binning (E):")
print(f"{'m':<8} {'<E>':<15} {'Error':<15}")
for m, em, err in zip(bin_sizes, energy_means, energy_errors):
    print(f"{m:<8} {em:<15.6f} {err:<15.6e}")

print(f"\nMagnetization binning (|M|):")
print(f"{'m':<8} {'<|M|>':<15} {'Error':<15}")
for m, mm, err in zip(bin_sizes_m, mag_means, mag_errors):
    print(f"{m:<8} {mm:<15.6f} {err:<15.6e}")


ValueError: could not convert string '-0.311927915' to int64 at row 0, column 1.